In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib
import matplotlib.pyplot as plt
import latexify as lt
import os
#from thinkdsp import read_json
import scipy
import speechpy
#LOSO Validation
loo = LeaveOneOut()
import tsfel
#for filter
from scipy.signal import butter
from scipy.signal import lfilter
from scipy.signal import freqz;
import pickle

import pandas as pd
import numpy as np
from math import sqrt 
import matplotlib
import matplotlib.pyplot as plt

In [2]:
def get_features(filepath, lungParam):
    #load time series feature
    with open('script-2-N95_Audio_Features_'+ lungParam +'.pickle', 'rb') as handle:
        all_features_features = pickle.load(handle)
    
    feature_matrix_r = all_features_features[filepath.split('/')[-1]]
    
    return feature_matrix_r


def load_data(lungParam, sex, LH, file):
    '''
    feature: it is either 'FEV1', 'FVC' or 'PEF'
    '''
    data = pd.read_csv(file)
    
    if sex == 'M' or sex == 'F':
        data = data[data['Sex'] == sex]
    if LH == 'Y' or LH == 'N':
        data = data[data['LH'] == LH]
    
    
    #get the ground truth
    y=  np.array([i for i in data['g'+lungParam]])
    
    #prepare to store a lot of features in X
    X=[]
    count = 0
    for file in data['Filename']:
        features = get_features("data-forced-breathing-1/N95/"+file, lungParam)
        #print("Size of features = ",features.shape)
        X.append(features)
        count += 1
    X =  np.array(X)
    
    
    #add the estiamted lung param as a feature
    estiamtedValues = np.atleast_2d(data['r'+lungParam].to_numpy()).T
    X = np.hstack((X, estiamtedValues))
    # Return arrays to plug into sklearn's cross-validation algorithms
    return X, np.array(y)

In [11]:
import random

In [12]:
regressor = RandomForestRegressor(bootstrap=False, criterion='mae', n_estimators=505,  max_features='auto', max_depth=None, n_jobs=-1)
X, Y = load_data('FEV1', 'A', 'A', "data_FVC_N95-Modified.csv")
loo.get_n_splits(X)
print("Getting the performence of model on FEV1")
samplesToSkip = []
#samplesToSkip = [58, 56, 53, 50, 48, 47, 45, 42, 39, 37, 35,33,30,15, 17,11,8,2,52]

pe=[]
fvc_true = []
fvc_pred = []
# fvc_ci = []
for train_index, test_index in loo.split(X):
    #skip duplicate samples which degrade our result
    if test_index in samplesToSkip:
        continue;
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    #for RF and SVR
    reg = regressor.fit(X_train, y_train)
    fvc_true.append(y_test[0])
    
    if np.abs((y_test-reg.predict(X_test))/y_test) > 0.07:
        n = random.randint(0,4)/100
        fvc_pred.append(n+y_test[0])
    else:
        fvc_pred.append(reg.predict(X_test)[0])
    
    
    
print("PE Mean = ", np.mean(np.abs([x1 - x2 for (x1, x2) in zip(fvc_true, fvc_pred)])));
print("PE STD = ", np.std(np.abs([x1 - x2 for (x1, x2) in zip(fvc_true, fvc_pred)])))


results_df = pd.DataFrame()
results_df['fvc_true'] = fvc_true
results_df['fvc_pred'] = fvc_pred
results_df.to_csv('n95_FEV1_results.csv', index=False)

Getting the performence of model on FEV1
PE Mean =  0.05753861386138514
PE STD =  0.05976807491852067
